Help functions

In [3]:
import numpy as np

def ptr(x, y, z):
    if max(x,y,z) == x:
        return 1
    if max(x,y,z) == y:
        return 2
    if max(x,y,z) == z:
        return 3

def g(x, y):
    if x == y:
        return 1
    else:
        return -1

def algo(dna1, dna2):
    F = np.zeros((len(dna1) + 1,len(dna2) + 1), dtype=int)
    Ptr = np.zeros((len(dna1) + 1,len(dna2) + 1), dtype=int)
    for i in range(1, len(dna1) + 1):
        F[i][0] = -i
    for i in range(1, len(dna2) + 1):
        F[0][i] = -i
    for i in range(1, len(dna1) + 1):
        for j in range(1, len(dna2) + 1):
            F[i][j] = max(F[i-1][j-1] + g(dna1[i - 1], dna2[j - 1]), F[i-1][j] - 1, F[i][j-1] - 1)
            Ptr[i][j] = ptr(F[i-1][j-1] + g(dna1[i - 1], dna2[j - 1]), F[i-1][j] - 1, F[i][j-1] - 1)
    i = len(dna1)
    j = len(dna2)
    res1 = []
    res2 = []
    while True:
        if Ptr[i][j] == 1:
            i -= 1
            j -= 1
            res1.append(i)
            res2.append(j)
        elif Ptr[i][j] == 2:
            i -= 1
            res1.append(i)
            res2.append(-1)
        elif Ptr[i][j] == 3:
            j -=1
            res1.append(-1)
            res2.append(j)
        elif Ptr[i][j] == 0 and i>0:
            i -= 1
            res1.append(i)
            res2.append(-1)
        elif Ptr[i][j] == 0 and j>0:
            j -= 1
            res1.append(-1)
            res2.append(j)
        else:
            break
    return [res1[::-1], res2[::-1]]

def get_residues(lines):
    resi = []
    resi_xyz = []
    count = 0
    s = []
    xyz = []
    for line in lines:
        toks = line.split()
        if len(toks) == 0:
            continue
        if toks[0] == 'ATOM' and toks[2] != 'H':
            if toks[5] != count:
                if s != []:
                    resi.append(s)
                    resi_xyz.append(xyz)
                    s = []
                    xyz = []
                count = toks[5]
            s.append(toks[2])
            xyz.append([float(toks[6]), float(toks[7]), float(toks[8])])
    if s != []:
        resi.append(s)
        resi_xyz.append(xyz)
    return [resi, resi_xyz]

def get_count(lines):
    resi, resi_xyz = get_residues(lines)
    count = sum([len(item) for item in resi])
    
    return count

RMSD

In [4]:
def rmsd(lines_1, lines_2):
    first, first_xyz  =  get_residues(lines_1)
    second, second_xyz = get_residues(lines_2)
        
    if len(first) != len(second):
        print("Different lengths")
        return 0
    
    rmsd = 0
    count = 0
    for k in range(0, len(first)):
        align = algo(first[k], second[k])
        d1 = align[0]
        d2 = align[1]
        i = 0
        j = 0
        while(i < len(d1) and j < len(d2)):
            if d1[i] == d2[j]:
                x = (float(first_xyz[k][d1[i]][0]) - float(second_xyz[k][d2[j]][0]))**2
                y = (float(first_xyz[k][d1[i]][1]) - float(second_xyz[k][d2[j]][1]))**2
                z = (float(first_xyz[k][d1[i]][2]) - float(second_xyz[k][d2[j]][2]))**2
                rmsd = rmsd + x + y + z
                count += 1
                i += 1
                j += 1
            elif d1[i] < d2[j]:
                i += 1
            elif d1[i] > d2[j]:
                j += 1
    return [count, (rmsd / count)**0.5]

RMSD between interface residues (interface residues: all atoms less 12A)

In [5]:
def rmsd_interface(lines_01, lines_02, lines_1, lines_2):
    first, first_xyz = get_residues(lines_01)
    second, second_xyz = get_residues(lines_02)
    
    resi_1 = interface(first, first_xyz, second, second_xyz)
    resi_2 = interface(second, second_xyz, first, first_xyz)
    for i in range(0, len(resi_2)):
        resi_1.append(len(first) + i)
        
    temp = first + second
    temp_xyz = first_xyz + second_xyz
    first = [temp[i] for i in resi_1]
    first_xyz = [temp_xyz[i] for i in resi_1]
    
    temp, temp_xyz = get_residues(lines_1 + lines_2)
    second = [temp[i] for i in resi_1]
    second_xyz = [temp_xyz[i] for i in resi_1]
    
    if len(first) != len(second):
        print("Different lengths")
        return 0
    
    rmsd = 0
    count = 0
    for k in range(0, len(first)):
        align = algo(first[k], second[k])
        d1 = align[0]
        d2 = align[1]
        i = 0
        j = 0
        while(i < len(d1) and j < len(d2)):
            if d1[i] == d2[j]:
                x = (float(first_xyz[k][d1[i]][0]) - float(second_xyz[k][d2[j]][0]))**2
                y = (float(first_xyz[k][d1[i]][1]) - float(second_xyz[k][d2[j]][1]))**2
                z = (float(first_xyz[k][d1[i]][2]) - float(second_xyz[k][d2[j]][2]))**2
                rmsd = rmsd + x + y + z
                count += 1
                i += 1
                j += 1
            elif d1[i] < d2[j]:
                i += 1
            elif d1[i] > d2[j]:
                j += 1
    return [count, (rmsd / count)**0.5]

Ligand RMSD (смещение между атомами лиганда)

In [6]:
def interface(first, first_xyz, second, second_xyz):
    result = []
    
    for i in range(0, len(first)):
        inter = False
        all_atoms = False
        for j in range(0, len(first[i])):
            for k in range(0, len(second)):
                for l in range(0, len(second[k])):
                    dist = ((first_xyz[i][j][0] - second_xyz[k][l][0])**2 + (first_xyz[i][j][1] - second_xyz[k][l][1])**2 + (first_xyz[i][j][2] - second_xyz[k][l][2])**2)**0.5
                    if dist < 12:
                        inter = True
                        break
                if inter:
                    break
            if not inter:
                break
            if j == len(first[i]) - 1:
                all_atoms = True
        if all_atoms:
            result.append(i)
    
    interface = []
    interface_xyz = []
    for i in range(0, len(result)):
        interface.append(first[result[i]])
        interface_xyz.append(first_xyz[result[i]])
    return result

Example

In [7]:
folder_1 = '1brs0A/'
folder_2 = '1brs0D/'
file_01 = open(folder_1 + folder_1[:-1] + '_0A.pdb')
pdb_01 = file_01.read()
lines_01 = pdb_01.split('\n')
file_02 = open(folder_2 + folder_2[:-1] + '_0A.pdb')
pdb_02 = file_02.read()
lines_02 = pdb_02.split('\n')

res_rmsd = []
res_rmsd_inter = []
res_rmsd_lig = []

for i in range(1, 7):
    file_1 = open(folder_1 + folder_1[:-1] + '_' + str(i) + 'A.pdb')
    file_2 = open(folder_2 + folder_2[:-1] + '_' + str(i) + 'A.pdb')
    pdb_1 = file_1.read()
    pdb_2 = file_2.read()
    lines_1 = pdb_1.split('\n')
    lines_2 = pdb_2.split('\n')
    res_rmsd.append(rmsd(lines_01 + lines_02, lines_1 + lines_2))
    res_rmsd_inter.append(rmsd_interface(lines_01, lines_02, lines_1, lines_2))
    if get_count(lines_01) > get_count(lines_02):
        res_rmsd_lig.append(rmsd(lines_02, lines_2))
    else:
        res_rmsd_lig(rmsd(lines_01, lines_1))
    
print("RMSD: ")
for i in range(0, len(res_rmsd)):
    print(str(i+1) + " " + str(res_rmsd[i][1]) + "\tATOMS:" + str(res_rmsd[i][0]))

print("\n")
print("Interface: ")
for i in range(0, len(res_rmsd_inter)):
    print(str(i+1) + " " + str(res_rmsd_inter[i][1]) + "\tATOMS:" + str(res_rmsd_inter[i][0]))

print("\n")
print("Ligand: ")
for i in range(0, len(res_rmsd)):
    print(str(i+1) + " " + str(res_rmsd_lig[i][1]) + "\tATOMS:" + str(res_rmsd_lig[i][0]))

RMSD: 
1 1.9083678255850711	ATOMS:1556
2 2.9365048054171203	ATOMS:1556
3 4.078948788972953	ATOMS:1556
4 4.751243001348947	ATOMS:1556
5 5.90256274367868	ATOMS:1556
6 7.755456893104413	ATOMS:1556


Interface: 
1 2.12860951206092	ATOMS:900
2 2.9555821496648798	ATOMS:900
3 3.5553710845884363	ATOMS:900
4 3.4469854873823973	ATOMS:900
5 5.1369564345267005	ATOMS:900
6 5.385427546888443	ATOMS:900


Ligand: 
1 2.1307725764100094	ATOMS:693
2 3.1498162838936588	ATOMS:693
3 3.8910628534721337	ATOMS:693
4 4.846001435051786	ATOMS:693
5 5.99336485872818	ATOMS:693
6 8.03466425727293	ATOMS:693
